In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src import configuration as config
from src.pipeline.pipeline_factory import PipelineFactory, ModelType, EvaluationType

from sklearn.tree import DecisionTreeRegressor
from category_encoders.one_hot import OneHotEncoder
from lightgbm import LGBMRegressor


In [9]:
# parameters={"splitter":["best","random"],
#             "max_depth" : [1,3,5,7,9,11,12],
#            "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
#            "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
#            "max_features":["auto","log2","sqrt",None],
#            "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] }

In [1]:
# LGBM-Regressor
param_grid = {
    'estimator__learning_rate': [0.1, 0.01, 0.001],
    'estimator__n_estimators': [100, 200, 500],
    'estimator__max_depth': [3, 5, 7],
    'estimator__num_leaves': [31, 50, 100],
    'estimator__subsample': [0.8, 1.0],
    'estimator__colsample_bytree': [0.8, 1.0]
}

In [4]:
# load the data
train_df = config.load_traindata_for_regression()
pipelineFactory = PipelineFactory()

# create the baseline pipeline
pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.REGRE_PREPROCESSED,
    verbose_level=1,
    evaluation=EvaluationType.GRID_SEARCH,
    param_grid=param_grid
)
pipeline.clear_steps()
pipeline.add_new_step(OneHotEncoder(), 'onehot')
# pipeline.change_estimator(DecisionTreeRegressor())
pipeline.change_estimator(LGBMRegressor())

pipeline.run()
scores = {}

Starting pipeline using method: EvaluationType.GRID_SEARCH
Performing grid search


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 324/324 [1:04:24<00:00, 11.93s/it]

Finished running the pipeline
Evaluation metrics:
    best_score: 0.3675
    best_params: {'estimator__learning_rate': 0.1, 'estimator__n_estimators': 500, 'estimator__max_depth': 7, 'estimator__num_leaves': 100, 'estimator__subsample': 0.8, 'estimator__colsample_bytree': 1.0}
